In [76]:
#imports to work with...
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from torch.utils.data import DataLoader
import torch
import torchvision
from torchvision import transforms
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import label_binarize

from cl_framework.continual_learning.metrics.metric_evaluator_incdec import MetricEvaluatorIncDec
from cl_framework.utilities.matrix_logger import IncDecLogger
from torchmetrics import Recall

In [77]:
# this is used later to create the class_to_idx
def kinetics_classes(classes_csv):
    df = pd.read_csv(classes_csv)
    classes_behaviors = {}

    for _, row in df.iterrows():
        class_name = row['Class']
        subcategory = row['Subcategory']
        
        # Check if the class_name is already in the dictionary, if not, create a new entry
        if class_name not in classes_behaviors:
            classes_behaviors[class_name] = []
        
        # Add the subcategory to the corresponding class_name entry in the dictionary
        classes_behaviors[class_name].append(subcategory)

    return classes_behaviors

In [78]:
#create a mapping between classes - behaviors
folder_csv = '../Kinetics/Info/'
class_csv = os.path.join(folder_csv, 'classes.csv')
classes_behaviors = kinetics_classes(class_csv)

#create a index for each class -- {class: idx}
class_to_idx = {key: i for i, key in enumerate(classes_behaviors.keys())}


In [79]:
predictions_names = ['food', 'phone','smoking','fatigue','selfcare']
targets_names = ['target_food','target_phone','target_smoking','target_fatigue','target_selfcare']

all_behaviors_dict = {
    'food': [
        'eating burger', 'eating cake', 'eating carrots', 'eating chips', 'eating doughnuts',
        'eating hotdog', 'eating ice cream', 'eating spaghetti', 'eating watermelon',
        'sucking lolly', 'tasting beer', 'tasting food', 'tasting wine', 'sipping cup'
    ],
    'phone': [
        'texting', 'talking on cell phone', 'looking at phone'
    ],
    'smoking': [
        'smoking', 'smoking hookah', 'smoking pipe'
    ],
    'fatigue': [
        'sleeping', 'yawning', 'headbanging', 'headbutting', 'shaking head'
    ],
    'selfcare': [
        'scrubbing face', 'putting in contact lenses', 'putting on eyeliner', 'putting on foundation',
        'putting on lipstick', 'putting on mascara', 'brushing hair', 'brushing teeth', 'braiding hair',
        'combing hair', 'dyeing eyebrows', 'dyeing hair'
    ]
    }


n_task = 6
total_classes = 5

In [80]:
results_path = '../runs_trainings/freeze_backbone/baseline_multilabel/weighted/new'
#criterion_type = 'multiclass'
criterion_type = 'multilabel'
seeds = [0,1,2]

In [81]:
def extract_data(task_data, metric_eval):
    predictions_list = []
    targets_list = []
    for i in range(len(predictions_names)):
        if criterion_type == 'multilabel':
            class_targets = task_data[targets_names[i]].tolist()
            targets_list.append(class_targets)
        class_predictions = task_data[predictions_names[i]].tolist()
        predictions_list.append(class_predictions)
    
    predictions = torch.Tensor(predictions_list).permute(1,0)
    if criterion_type == 'multilabel':
        targets = torch.Tensor(targets_list).permute(1,0)
    else:
        targets = torch.Tensor(task_data['target'])

    subcategory = task_data['subcategory']
    data_path = task_data['video_path']

    if criterion_type == 'multilabel':
        binarized_targets = targets
        targets = torch.Tensor(torch.argmax(targets, axis=1))
    else:
        binarized_targets = torch.Tensor(label_binarize(targets, classes=[i for i in range(5)]))

    
    metric_eval.update(targets, binarized_targets, predictions, subcategory, data_path)

In [82]:
for idx_seed in range(len(seeds)):

    seed_path = os.path.join(results_path, 'seed_' + str(seeds[idx_seed]))
        
    for exp_dir in os.listdir(seed_path):
        exp_path = os.path.join(seed_path,exp_dir)
        error_analysis_csv_path = os.path.join(exp_path,'error_analysis')

        logger_dir = os.path.join(exp_path,'logger')
        validation_logger_dir = os.path.join(exp_path,'validation_logger')

        old_logger_dir = os.path.join(logger_dir,'old')
        if not os.path.exists(old_logger_dir):
            os.mkdir(old_logger_dir)
        old_validation_logger_dir = os.path.join(validation_logger_dir,'old')
        if not os.path.exists(old_validation_logger_dir):
            os.mkdir(old_validation_logger_dir)

        # move files in old directories
        for file in os.listdir(logger_dir):
            if '.out' in file:
                os.rename(os.path.join(logger_dir,file),os.path.join(old_logger_dir,file))

        for file in os.listdir(validation_logger_dir):
            if '.out' in file:
                os.rename(os.path.join(validation_logger_dir,file),os.path.join(old_validation_logger_dir,file))

In [83]:
task_dict = None
for idx_seed in range(len(seeds)):

    seed_path = os.path.join(results_path, 'seed_' + str(seeds[idx_seed]))
        
    for exp_dir in os.listdir(seed_path):
        exp_path = os.path.join(seed_path,exp_dir)
        error_analysis_csv_path = os.path.join(exp_path,'error_analysis')

        
        
        logger = IncDecLogger(out_path=exp_path, n_task=n_task, task_dict=task_dict, all_behaviors_dict = all_behaviors_dict, class_to_idx= class_to_idx, num_classes=total_classes, criterion_type=criterion_type, print_log_files=False)
        for i in range(n_task):
            task_csv = error_analysis_csv_path + '/task_{}_test_error_analysis.csv'.format(str(i))
            task_data = pd.read_csv(task_csv)
            metric_eval = MetricEvaluatorIncDec('../random_tries/', num_classes=total_classes, criterion_type=criterion_type, all_behaviors_dict=all_behaviors_dict, class_to_idx=class_to_idx)
            extract_data(task_data,metric_eval)
            acc, ap, acc_per_class, mean_ap, map_weighted, precision_per_class, recall_per_class, exact_match, ap_per_subcategory, recall_per_subcategory, accuracy_per_subcategory, precision_per_subcategory = metric_eval.get(verbose=True)
            logger.update_accuracy(current_training_task_id=i, acc_value=acc, ap_value=ap,
                                acc_per_class=acc_per_class, mean_ap=mean_ap, map_weighted=map_weighted, 
                                precision_per_class=precision_per_class, recall_per_class=recall_per_class, 
                                exact_match=exact_match, ap_per_subcategory=ap_per_subcategory, recall_per_subcategory=recall_per_subcategory, 
                                accuracy_per_subcategory=accuracy_per_subcategory, precision_per_subcategory=precision_per_subcategory)
            logger.update_forgetting(current_training_task_id=i)
            logger.print_latest(current_training_task_id=i)
        logger.compute_average()
        logger.print_file()


        val_logger = IncDecLogger(out_path=exp_path, n_task=n_task, task_dict=task_dict, all_behaviors_dict = all_behaviors_dict, class_to_idx= class_to_idx, num_classes=total_classes, criterion_type=criterion_type, print_log_files=False,validation_mode=True)
        for i in range(n_task):
            task_csv = error_analysis_csv_path + '/task_{}_validation_error_analysis.csv'.format(str(i))
            task_data = pd.read_csv(task_csv)
            metric_eval = MetricEvaluatorIncDec('../random_tries/', num_classes=total_classes, criterion_type=criterion_type, all_behaviors_dict=all_behaviors_dict, class_to_idx=class_to_idx)
            extract_data(task_data,metric_eval)
            acc, ap, acc_per_class, mean_ap, map_weighted, precision_per_class, recall_per_class, exact_match, ap_per_subcategory, recall_per_subcategory, accuracy_per_subcategory, precision_per_subcategory = metric_eval.get(verbose=True)
            val_logger.update_accuracy(current_training_task_id=i, acc_value=acc, ap_value=ap, 
                                acc_per_class=acc_per_class, mean_ap=mean_ap, map_weighted=map_weighted, 
                                precision_per_class=precision_per_class, recall_per_class=recall_per_class, 
                                exact_match=exact_match, ap_per_subcategory=ap_per_subcategory, recall_per_subcategory=recall_per_subcategory, 
                                accuracy_per_subcategory=accuracy_per_subcategory, precision_per_subcategory=precision_per_subcategory)
            val_logger.update_forgetting(current_training_task_id=i)
            val_logger.print_latest(current_training_task_id=i)
        val_logger.compute_average()
        val_logger.print_file()

 - task accuracy: 0.42162162162162165
 - task average precision: tensor([0.5699, 0.1701, 0.1477, 0.2524, 0.6790])
 - task acc per class: [0.6621621621621622, 0.8513513513513513, 0.772972972972973, 0.8216216216216217, 0.7351351351351352]
 - task precision per class: tensor([0.5824, 0.1622, 0.1447, 0.3182, 0.5902])
 - task recall per class: tensor([0.3786, 0.2000, 0.3667, 0.2800, 0.6000])
 - task mAP: 0.3638175129890442
 - task weighted mAP: 0.49573779106140137

 >>> Test on task  0 : acc= 42.2%,  |  <<<
 - task accuracy: 0.3783783783783784
 - task average precision: tensor([0.5756, 0.1489, 0.1489, 0.2383, 0.6832])
 - task acc per class: [0.6567567567567567, 0.845945945945946, 0.6945945945945946, 0.8216216216216217, 0.7378378378378379]
 - task precision per class: tensor([0.5684, 0.2000, 0.1327, 0.1923, 0.6264])
 - task recall per class: tensor([0.3857, 0.3000, 0.5000, 0.1000, 0.4750])
 - task mAP: 0.35897547006607056
 - task weighted mAP: 0.4957229197025299

 >>> Test on task  1 : acc= 